# Chapter 4: GPS - The General Problem Solver

In [2]:
(load "utils.lisp")
(use-package :utils)

T

T

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining UTILS:MAPPEND in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining UTILS:CROSS-PRODUCT in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining UTILS:COMBINE-ALL in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining UTILS:FIND-ALL in DEFUN


In [3]:
(defvar *current* nil "the current states of world")
(defvar *ops* '() "list of available options")

(defstruct op
    "An operation"
    action preconds add-list del-list)

(defun GPS (*current* goals *ops*)
    (if (every #'achieve goals) 'solved))

(defun achieve (goal) 
    (or (member goal *current*)
        (some #'apply-op (find-all goal *ops* :test #'appropriate-p))))

(defun appropriate-p (goal op)
    "An op is appropriate if the goal is in its add-list."
    (member goal (op-add-list op)))

(defun apply-op (op)
    "Apply `op` to the current world state."
    (when (every #'achieve (op-preconds op))   ;; achieve check other operation recursively
        (print (list 'executing (op-action op)))
        (setf *current* (set-difference (union (op-add-list op) *current*) (op-del-list op)))
        ; (format t "op:~a preconditions ~a are not met" (op-action op) (op-preconds op))
        t))

(defparameter *school-ops*
  (list
    (make-op :action 'drive-son-to-school
         :preconds '(son-at-home car-works)
         :add-list '(son-at-school)
         :del-list '(son-at-home))
    (make-op :action 'shop-installs-battery
         :preconds '(car-needs-battery shop-knows-problem shop-has-money)
         :add-list '(car-works))
    (make-op :action 'tell-shop-problem
         :preconds '(in-communication-with-shop)
         :add-list '(shop-knows-problem))
    (make-op :action 'telephone-shop
         :preconds '(know-phone-number)
         :add-list '(in-communication-with-shop))
    (make-op :action 'look-up-number
         :preconds '(have-phone-book)
         :add-list '(know-phone-number))
    (make-op :action 'give-shop-money
         :preconds '(have-money)
         :add-list '(shop-has-money)
         :del-list '(have-money))))

*CURRENT*

*OPS*

OP

GPS

ACHIEVE

APPROPRIATE-P

APPLY-OP

*SCHOOL-OPS*

SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::ACHIEVE
SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::APPLY-OP
SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::APPROPRIATE-P


In [4]:
(gps '(son-at-home car-needs-battery have-money have-phone-book)
    '(son-at-school)
    *school-ops*)

SOLVED


(EXECUTING LOOK-UP-NUMBER) 
(EXECUTING TELEPHONE-SHOP) 
(EXECUTING TELL-SHOP-PROBLEM) 
(EXECUTING GIVE-SHOP-MONEY) 
(EXECUTING SHOP-INSTALLS-BATTERY) 
(EXECUTING DRIVE-SON-TO-SCHOOL) 

In [5]:
;; the phone book look up is executed because in the preconds, things are evaluated left to right
;; => '(car-needs-battery shop-knows-problem shop-has-money)
(gps '(son-at-home car-needs-battery have-phone-book)
    '(son-at-school)
    *school-ops*)

NIL


(EXECUTING LOOK-UP-NUMBER) 
(EXECUTING TELEPHONE-SHOP) 
(EXECUTING TELL-SHOP-PROBLEM) 

In [6]:
;; Problem 2: competing goals
(defun achieve-all (goals)
    (and (every #'achieve goals) (subsetp goals *current*)))

(defun GPS (*current* goals *ops*)
    (if (achieve-all goals) 'solved))

(gps '(son-at-home car-needs-battery have-money have-phone-book)
    '(have-money son-at-school)
    *school-ops*)

ACHIEVE-ALL

GPS

NIL

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::GPS in DEFUN

(EXECUTING LOOK-UP-NUMBER) 
(EXECUTING TELEPHONE-SHOP) 
(EXECUTING TELL-SHOP-PROBLEM) 
(EXECUTING GIVE-SHOP-MONEY) 
(EXECUTING SHOP-INSTALLS-BATTERY) 
(EXECUTING DRIVE-SON-TO-SCHOOL) 

In [7]:
;; problem 4: recursive subgoal problem
(push (make-op :action 'ask-phone-number
      :preconds '(in-communication-with-shop)
      :add-list '(know-phone-number))
    *school-ops*)

; (gps '(son-at-home car-needs-battery have-money)
; '(son-at-school)
; *school-ops*)

(#S(OP
    :ACTION ASK-PHONE-NUMBER
    :PRECONDS (IN-COMMUNICATION-WITH-SHOP)
    :ADD-LIST (KNOW-PHONE-NUMBER)
    :DEL-LIST NIL)
 #S(OP
    :ACTION DRIVE-SON-TO-SCHOOL
    :PRECONDS (SON-AT-HOME CAR-WORKS)
    :ADD-LIST (SON-AT-SCHOOL)
    :DEL-LIST (SON-AT-HOME))
 #S(OP
    :ACTION SHOP-INSTALLS-BATTERY
    :PRECONDS (CAR-NEEDS-BATTERY SHOP-KNOWS-PROBLEM SHOP-HAS-MONEY)
    :ADD-LIST (CAR-WORKS)
    :DEL-LIST NIL)
 #S(OP
    :ACTION TELL-SHOP-PROBLEM
    :PRECONDS (IN-COMMUNICATION-WITH-SHOP)
    :ADD-LIST (SHOP-KNOWS-PROBLEM)
    :DEL-LIST NIL)
 #S(OP
    :ACTION TELEPHONE-SHOP
    :PRECONDS (KNOW-PHONE-NUMBER)
    :ADD-LIST (IN-COMMUNICATION-WITH-SHOP)
    :DEL-LIST NIL)
 #S(OP
    :ACTION LOOK-UP-NUMBER
    :PRECONDS (HAVE-PHONE-BOOK)
    :ADD-LIST (KNOW-PHONE-NUMBER)
    :DEL-LIST NIL)
 #S(OP
    :ACTION GIVE-SHOP-MONEY
    :PRECONDS (HAVE-MONEY)
    :ADD-LIST (SHOP-HAS-MONEY)
    :DEL-LIST (HAVE-MONEY)))

In [8]:
;; Build debugging tools
(defvar *dbg-ids* nil "identifiers used by `dbg`")

(defun dbg (id fmt-string &rest fmt-args)
    (when (member id *dbg-ids*)
        (fresh-line *debug-io*)
        (format *debug-io* fmt-string fmt-args)
        t))

(defun dbg-indent (id indent fmt-string &rest fmt-args)
  "Print indented debugging info if (DEBUG ID) has been specified."
  (when (member id *dbg-ids*)
    (fresh-line *debug-io*)
    (dotimes (i indent) (princ " " *debug-io*))
    (format *debug-io* fmt-string fmt-args)))

(defun debug1 (&rest ids)
    "turn on `dbg` for id"
    (setq *dbg-ids* (union ids *dbg-ids*)))

(defun undebug1 (&rest ids)
    "turn off `dbg` for id"
    (setq *dbg-ids* 
        (if (null ids) nil (set-difference *dbg-ids* ids))))

;; testing code
(debug1 :gps)
(dbg :gps "this is ~a" '(1 2 3))
(dbg-indent :gps 2 "this is ~a" '(1 2 3))
(undebug1)

*DBG-IDS*

DBG

DBG-INDENT

DEBUG1

UNDEBUG1

(:GPS)

T

NIL

NIL


this is ((1 2 3))
  this is ((1 2 3))

In [9]:
;; Iterate on version 2
(defun starts-with (list x)
    (and (consp list) (eql (first list) x)))

(defun executing-p (x)
    "Is x of the form (executing ...)"
    (starts-with x 'executing))

(defun convert-op (op)
    "Make op conform to (executing op) convention"
    (unless (some #'executing-p (op-add-list op))
        (push (list 'executing (op-action op)) (op-add-list op)))
        op)

;; defines (executing action) as a state as well
(defun op (action &key preconds add-list del-list)
    (convert-op
        (make-op :action action :preconds preconds
        :add-list add-list :del-list del-list)))

;; update the existing data structure
(mapc #'convert-op *school-ops*)

STARTS-WITH

EXECUTING-P

CONVERT-OP

OP

(#S(OP
    :ACTION ASK-PHONE-NUMBER
    :PRECONDS (IN-COMMUNICATION-WITH-SHOP)
    :ADD-LIST ((EXECUTING ASK-PHONE-NUMBER) KNOW-PHONE-NUMBER)
    :DEL-LIST NIL)
 #S(OP
    :ACTION DRIVE-SON-TO-SCHOOL
    :PRECONDS (SON-AT-HOME CAR-WORKS)
    :ADD-LIST ((EXECUTING DRIVE-SON-TO-SCHOOL) SON-AT-SCHOOL)
    :DEL-LIST (SON-AT-HOME))
 #S(OP
    :ACTION SHOP-INSTALLS-BATTERY
    :PRECONDS (CAR-NEEDS-BATTERY SHOP-KNOWS-PROBLEM SHOP-HAS-MONEY)
    :ADD-LIST ((EXECUTING SHOP-INSTALLS-BATTERY) CAR-WORKS)
    :DEL-LIST NIL)
 #S(OP
    :ACTION TELL-SHOP-PROBLEM
    :PRECONDS (IN-COMMUNICATION-WITH-SHOP)
    :ADD-LIST ((EXECUTING TELL-SHOP-PROBLEM) SHOP-KNOWS-PROBLEM)
    :DEL-LIST NIL)
 #S(OP
    :ACTION TELEPHONE-SHOP
    :PRECONDS (KNOW-PHONE-NUMBER)
    :ADD-LIST ((EXECUTING TELEPHONE-SHOP) IN-COMMUNICATION-WITH-SHOP)
    :DEL-LIST NIL)
 #S(OP
    :ACTION LOOK-UP-NUMBER
    :PRECONDS (HAVE-PHONE-BOOK)
    :ADD-LIST ((EXECUTING LOOK-UP-NUMBER) KNOW-PHONE-NUMBER)
    :DEL-LIST NIL)
 #S(OP
    :ACTI

In [10]:
;; Major updates:
;; - State changed from unordered set to ordered list, including '(start) and '(executing <action>)
;; - Leverage function stack to track states, local value, function return signature

(defun GPS (state goals &optional (*ops* *ops*))  ;; dynamic scopping binding, extent the life for *ops* while GPS is executing
  "General Problem Solver: from state, achieve goals using *ops*."
  (remove-if #'atom (achieve-all (cons '(start) state) goals nil)))

(defun achieve (state goal goal-stack)
  "A goal is achieved if it already holds,
  or if there is an appropriate op for it that is applicable."
  (dbg-indent :gps (length goal-stack) "Goal: ~a" goal)
  (cond ((member-equal goal state) state)
      ((member-equal goal goal-stack) nil)  ;; fail and detect recursive goal
      (t (some #'(lambda (op) (apply-op state goal op goal-stack))
          (find-all goal *ops* :test #'appropriate-p)))))

(defun achieve-all (state goals goal-stack)
  "Achieve each goal, and make sure they still hold at the end."
  (let ((current-state state))
    (if (and (every #'(lambda (g)
            (setf current-state
              (achieve current-state g goal-stack)))
          goals)
        (subsetp goals current-state :test #'equal))
      current-state)))

  ;; #'equal checks the value, because now a condition could be non-atom
  (defun member-equal (item list)
    (member item list :test #'equal))  

(defun apply-op (state goal op goal-stack)
  "Return a new, transformed state if op is applicable."
  (dbg-indent :gps (length goal-stack) "Consider: ~a" (op-action op))
  (let ((state2 (achieve-all state (op-preconds op)
            (cons goal goal-stack))))
    (unless (null state2)
      ;; Return an updated state
      (dbg-indent :gps (length goal-stack) "Action: ~a" (op-action op))
      (append (remove-if #'(lambda (x)
            (member-equal x (op-del-list op)))
          state2)
        (op-add-list op)))))

(defun appropriate-p (goal op)
  "An op is appropriate to a goal if it is in its add-list."
  (member-equal goal (op-add-list op)))

(defun use (oplist)
  "Use oplist as the default list of operators."
  ;; Return something useful, but not too verbose:
  ;; the number of operators.
   (length (setf *ops* oplist)))

GPS

ACHIEVE

ACHIEVE-ALL

MEMBER-EQUAL

APPLY-OP

APPROPRIATE-P

USE

SB-INT:SIMPLE-STYLE-WARNING: The function ACHIEVE-ALL is called with three arguments, but wants exactly one.
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::GPS in DEFUN
SB-INT:SIMPLE-STYLE-WARNING: The function APPLY-OP is called with four arguments, but wants exactly one.
SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::MEMBER-EQUAL
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::ACHIEVE in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::ACHIEVE-ALL in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::APPLY-OP in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::APPROPRIATE-P in DEFUN


In [11]:
(use *school-ops*)

(gps '(son-at-home car-needs-battery have-money have-phone-book)
      '(son-at-school))

7

((START) (EXECUTING LOOK-UP-NUMBER) (EXECUTING TELEPHONE-SHOP)
 (EXECUTING TELL-SHOP-PROBLEM) (EXECUTING GIVE-SHOP-MONEY)
 (EXECUTING SHOP-INSTALLS-BATTERY) (EXECUTING DRIVE-SON-TO-SCHOOL))

In [12]:
(debug1 :gps)
(gps '(son-at-home car-needs-battery have-money have-phone-book)
      '(son-at-school))
(undebug1)

(:GPS)

((START) (EXECUTING LOOK-UP-NUMBER) (EXECUTING TELEPHONE-SHOP)
 (EXECUTING TELL-SHOP-PROBLEM) (EXECUTING GIVE-SHOP-MONEY)
 (EXECUTING SHOP-INSTALLS-BATTERY) (EXECUTING DRIVE-SON-TO-SCHOOL))

NIL

Goal: (SON-AT-SCHOOL)
Consider: (DRIVE-SON-TO-SCHOOL)
 Goal: (SON-AT-HOME)
 Goal: (CAR-WORKS)
 Consider: (SHOP-INSTALLS-BATTERY)
  Goal: (CAR-NEEDS-BATTERY)
  Goal: (SHOP-KNOWS-PROBLEM)
  Consider: (TELL-SHOP-PROBLEM)
   Goal: (IN-COMMUNICATION-WITH-SHOP)
   Consider: (TELEPHONE-SHOP)
    Goal: (KNOW-PHONE-NUMBER)
    Consider: (ASK-PHONE-NUMBER)
     Goal: (IN-COMMUNICATION-WITH-SHOP)
    Consider: (LOOK-UP-NUMBER)
     Goal: (HAVE-PHONE-BOOK)
    Action: (LOOK-UP-NUMBER)
   Action: (TELEPHONE-SHOP)
  Action: (TELL-SHOP-PROBLEM)
  Goal: (SHOP-HAS-MONEY)
  Consider: (GIVE-SHOP-MONEY)
   Goal: (HAVE-MONEY)
  Action: (GIVE-SHOP-MONEY)
 Action: (SHOP-INSTALLS-BATTERY)
Action: (DRIVE-SON-TO-SCHOOL)

In [13]:
;; It avoids problems in version 1

;; Achieve both goals via `achieve-all` and comparison in the end 
(gps '(son-at-home car-needs-battery have-money have-phone-book)
      '(have-money son-at-school))

;; Leap before look problem is solved; still thinking ahead, but doesn't return the execution
(gps '(son-at-home car-needs-battery have-money have-phone-book)
      '(son-at-school have-money))

;; infinite recursion, exit as soon as loop being detected
(gps '(son-at-home car-needs-battery have-money)
      '(son-at-school) )

NIL

NIL

NIL

In [14]:
;; simple test case is still valid
(gps '(son-at-home) '(son-at-home))

((START))

In [15]:
;; Apply to new problem

;; State summary: 
;; - at-<location|object>
;; - chair-at-<location>
(defparameter *banana-ops*
  (list
    (op
      'climb-on-chair
      :preconds '(chair-at-middle-room at-middle-room on-floor)
      :add-list '(at-bananas on-chair)
      :del-list '(at-middle-room on-floor))
    (op
      'push-chair-from-door-to-middle-room
      :preconds '(chair-at-door at-door)
      :add-list '(chair-at-middle-room at-middle-room)
      :del-list '(chair-at-door at-door))
    (op
      'walk-from-door-to-middle-room
      :preconds '(at-door on-floor)
      :add-list '(at-middle-room)
      :del-list '(at-door))
    (op
      'grasp-bananas
      :preconds '(at-bananas empty-handed)
      :add-list '(has-bananas)
      :del-list '(empty-handed))
    (op
      'drop-ball
      :preconds '(has-ball)
      :add-list '(empty-handed)
      :del-list '(has-ball))
    (op
      'eat-bananas
      :preconds '(has-bananas)
      :add-list '(empty-handed not-hungry)
      :del-list '(has-bananas hungry))))

(use *banana-ops*)
; (debug1 :gps)
(gps '(chair-at-door at-door has-ball hungry on-floor) '(not-hungry))

*BANANA-OPS*

6

((START) (EXECUTING PUSH-CHAIR-FROM-DOOR-TO-MIDDLE-ROOM)
 (EXECUTING CLIMB-ON-CHAIR) (EXECUTING DROP-BALL) (EXECUTING GRASP-BANANAS)
 (EXECUTING EAT-BANANAS))

In [17]:
;; Try again with maze search!

;; Be specific about our definition of actions
(defun GPS (state goals &optional (*ops* *ops*))
  "General Problem Solver: from state, achieve goals using *ops*."
  (find-all-if #'action-p
        (achieve-all (cons '(start) state) goals nil)))

(defun action-p (x)
  "Is x something that is (start) or (executing ...)?"
  (or (equal x '(start)) (executing-p x)))

(defun make-maze-ops (pair)
    "make maze ops in both direction."
    (list (make-maze-op (first pair) (second pair))
          (make-maze-op (second pair) (first pair))))

(defun make-maze-op (here there)
    (op
        `(move from ,here to ,there)   ;; use of comma unquote inside back quote
        :preconds `((at ,here))
        :add-list `((at ,there))
        :del-list `((at ,here))))

(defparameter *maze-ops*
  (mappend #'make-maze-ops
    '((1 2) (2 3) (3 4) (4 9) (9 14) (9 8) (8 7) (7 12) (12 13)
      (12 11) (11 6) (11 16) (16 17) (17 22) (21 22) (22 23)
      (23 18) (23 24) (24 19) (19 20) (20 15) (15 10) (10 5) (20 25))))

(use *maze-ops*)
; (debug1 :gps)
(gps '((at 1)) '((at 25)))

GPS

ACTION-P

MAKE-MAZE-OPS

MAKE-MAZE-OP

*MAZE-OPS*

48

((START) (EXECUTING (MOVE FROM 1 TO 2)) (EXECUTING (MOVE FROM 2 TO 3))
 (EXECUTING (MOVE FROM 3 TO 4)) (EXECUTING (MOVE FROM 4 TO 9))
 (EXECUTING (MOVE FROM 9 TO 8)) (EXECUTING (MOVE FROM 8 TO 7))
 (EXECUTING (MOVE FROM 7 TO 12)) (EXECUTING (MOVE FROM 12 TO 11))
 (EXECUTING (MOVE FROM 11 TO 16)) (EXECUTING (MOVE FROM 16 TO 17))
 (EXECUTING (MOVE FROM 17 TO 22)) (EXECUTING (MOVE FROM 22 TO 23))
 (EXECUTING (MOVE FROM 23 TO 24)) (EXECUTING (MOVE FROM 24 TO 19))
 (EXECUTING (MOVE FROM 19 TO 20)) (EXECUTING (MOVE FROM 20 TO 25)))

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::GPS in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::ACTION-P in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::MAKE-MAZE-OPS in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::MAKE-MAZE-OP in DEFUN


In [24]:
;; The returned result could be extracted out
(defun find-path (start end)
    (let ((results (gps '((at 1)) '((at 25)))))
        (unless (null results)
            (cons start (mapcar #'destination (remove '(start) results :test #'equal))))))

(defun destination (action)
    (fifth (second action)))

(find-path 1 25)

FIND-PATH

DESTINATION

(1 2 3 4 9 8 7 12 11 16 17 22 23 24 19 20 25)

SB-INT:SIMPLE-STYLE-WARNING: The variable END is defined but never used.
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::FIND-PATH in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::DESTINATION in DEFUN


In [29]:
;; Apply to block world domain

;; Summary of possible state
;; - block_i on block_j
;; - block_i's top is empty
;; special block 'table

(defun move-ons (a b c)
    "Returns state after moves a from b to c"
    (if (eq b 'table)
        `((,a on ,c))
        `((,a on ,c) (space on ,b))))

(defun move-op (a b c)
    "The action to move a from b to c"
    (op
        `(move ,a from ,b to ,c)
        :preconds `((space on ,c) (,a on ,b) (space on ,a))
        :add-list (move-ons a b c)
        :del-list (move-ons a c b)))

(defun make-block-ops (blocks)
    (let ((ops nil))
        (dolist (a blocks)
            (dolist (b blocks)
                (unless (equal a b)
                    (dolist (c blocks)
                        (unless (or (equal c b) (equal c a))
                            (push (move-op a b c) ops)))   ;; move between any three blocks
                    (push (move-op a 'table b) ops)   ;; move between table and another block
                    (push (move-op a b 'table) ops))))
        ops))

MOVE-ONS

MOVE-OP

MAKE-BLOCK-OPS

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::MOVE-ONS in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::MOVE-OP in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::MAKE-BLOCK-OPS in DEFUN


In [31]:
(use (make-block-ops '(a b)))
(gps '((a on table) (b on table) (space on a) (space on b) (space on table))
    '((a on b) (b on table)))

4

((START) (EXECUTING (MOVE A FROM TABLE TO B)))

In [51]:
;; invert the block
; (debug1 :gps)
(gps '((a on b) (b on table) (space on a) (space on table))
    '((b on a)))

((START) (EXECUTING (MOVE A FROM B TO TABLE))
 (EXECUTING (MOVE B FROM TABLE TO A)))

In [52]:
;; invert three blocks
(use (make-block-ops '(a b c)))
(gps '((a on b) (b on c) (c on table) (space on a) (space on table))
    '((b on a) (c on b)))

;; oops, prerequisite clobbers sibling goal" situation again, but not recovered
(gps '((a on b) (b on c) (c on table) (space on a) (space on table))
      '((c on b) (b on a)))

18

((START) (EXECUTING (MOVE A FROM B TO TABLE)) (EXECUTING (MOVE B FROM C TO A))
 (EXECUTING (MOVE C FROM TABLE TO B)))

((START) (EXECUTING (MOVE A FROM B TO TABLE)) (EXECUTING (MOVE B FROM C TO A))
 (EXECUTING (MOVE C FROM TABLE TO B)))

In [54]:
(defun achieve-all (state goals goal-stack)
    "Achieve all goals in some orderings"
    (some #'(lambda (goals) (achieve-each state goals goal-stack))
        (orderings-simple goals)))

(defun orderings-simple (l)
    "Consider two possible orderings of list"
    (if (> (length l) 1)
        (list l (reverse l))
        (list l)))

(defun achieve-each (state goals goal-stack)
  "Achieve each goal, and make sure they still hold at the end."
  (let ((current-state state))
    (if (and (every #'(lambda (g)
            (setf current-state
              (achieve current-state g goal-stack)))
          goals)
        (subsetp goals current-state :test #'equal))
      current-state)))

(gps '((a on b) (b on c) (c on table) (space on a) (space on table))
      '((c on b) (b on a)))

ACHIEVE-ALL

ORDERINGS-SIMPLE

ACHIEVE-EACH

((START) (EXECUTING (MOVE A FROM B TO TABLE)) (EXECUTING (MOVE B FROM C TO A))
 (EXECUTING (MOVE C FROM TABLE TO B)))

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::ACHIEVE-ALL in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::ORDERINGS-SIMPLE in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::ACHIEVE-EACH in DEFUN


In [59]:
;; Find efficient solutions
(gps '((c on a) (a on table) (b on table)
      (space on c) (space on b) (space on table))
    '((c on table) (a on b)))

;; Change order of operation tried
(defun achieve (state goal goal-stack)
  "A goal is achieved if it already holds,
  or if there is an appropriate op for it that is applicable."
  (dbg-indent :gps (length goal-stack) "Goal:~a" goal)
  (cond ((member-equal goal state) state)
      ((member-equal goal goal-stack) nil)
      (t (some #'(lambda (op) (apply-op state goal op goal-stack))
          (appropriate-ops goal state))))) ;***

(defun appropriate-ops (goal state)
  "Return a list of appropriate operators,
  sorted by the number of unfulfilled preconditions."
  (sort (copy-list (find-all goal *ops* :test #'appropriate-p)) #'<
      :key #'(lambda (op)
          (count-if #'(lambda (precond) (not (member-equal precond state)))
            (op-preconds op)))))

(gps '((a on b) (b on c) (c on table) (space on a) (space on table))
      '((c on b) (b on a)))

((START) (EXECUTING (MOVE C FROM A TO TABLE))
 (EXECUTING (MOVE A FROM TABLE TO B)))

ACHIEVE

APPROPRIATE-OPS

((START) (EXECUTING (MOVE A FROM B TO TABLE)) (EXECUTING (MOVE B FROM C TO A))
 (EXECUTING (MOVE C FROM TABLE TO B)))

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::ACHIEVE in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::APPROPRIATE-OPS in DEFUN


In [93]:
;; exercise 4.2 permutation generator
(defun perm (l)
    (cond ((null l) '(()))
        (t (let ((results '()))
            (dotimes (i (length l))
                ;; Take nth element out from the list, then concatenate on the permutation of the rest
                (setf results (append results
                    (mapcar #'(lambda (rl) (cons (nth i l) rl))
                        (perm (append (subseq l 0 i) (subseq l (+ 1 i) (length l)))))))) 
            results))))

(perm '(1 2 3 4 5))





PERM

((1 2 3 4 5) (1 2 3 5 4) (1 2 4 3 5) (1 2 4 5 3) (1 2 5 3 4) (1 2 5 4 3)
 (1 3 2 4 5) (1 3 2 5 4) (1 3 4 2 5) (1 3 4 5 2) (1 3 5 2 4) (1 3 5 4 2)
 (1 4 2 3 5) (1 4 2 5 3) (1 4 3 2 5) (1 4 3 5 2) (1 4 5 2 3) (1 4 5 3 2)
 (1 5 2 3 4) (1 5 2 4 3) (1 5 3 2 4) (1 5 3 4 2) (1 5 4 2 3) (1 5 4 3 2)
 (2 1 3 4 5) (2 1 3 5 4) (2 1 4 3 5) (2 1 4 5 3) (2 1 5 3 4) (2 1 5 4 3)
 (2 3 1 4 5) (2 3 1 5 4) (2 3 4 1 5) (2 3 4 5 1) (2 3 5 1 4) (2 3 5 4 1)
 (2 4 1 3 5) (2 4 1 5 3) (2 4 3 1 5) (2 4 3 5 1) (2 4 5 1 3) (2 4 5 3 1)
 (2 5 1 3 4) (2 5 1 4 3) (2 5 3 1 4) (2 5 3 4 1) (2 5 4 1 3) (2 5 4 3 1)
 (3 1 2 4 5) (3 1 2 5 4) (3 1 4 2 5) (3 1 4 5 2) (3 1 5 2 4) (3 1 5 4 2)
 (3 2 1 4 5) (3 2 1 5 4) (3 2 4 1 5) (3 2 4 5 1) (3 2 5 1 4) (3 2 5 4 1)
 (3 4 1 2 5) (3 4 1 5 2) (3 4 2 1 5) (3 4 2 5 1) (3 4 5 1 2) (3 4 5 2 1)
 (3 5 1 2 4) (3 5 1 4 2) (3 5 2 1 4) (3 5 2 4 1) (3 5 4 1 2) (3 5 4 2 1)
 (4 1 2 3 5) (4 1 2 5 3) (4 1 3 2 5) (4 1 3 5 2) (4 1 5 2 3) (4 1 5 3 2)
 (4 2 1 3 5) (4 2 1 5 3) (4 2 3 1 5) (4 2 3 5 1) (4

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::PERM in DEFUN


In [97]:
;; more elegant solution using 'mapcan and (remove :count 1)
(defun permutations (bag)
  "Return a list of all the permutations of the input."
  ;; If the input is nil, there is only one permutation:
  ;; nil itself
  (if (null bag)
      '(())
      ;; Otherwise, take an element, e, out of the bag
      ;; Generate all permutations of the remaining elements,
      ;; And add e to the front of each of these.
      ;; Do this for all possible e to generate all permutations,
      (mapcan #'(lambda (e)
          (mapcar #'(lambda (p) (cons e p))
            (permutations
              (remove e bag :count 1 :test #'eq))))
        bag)))

(permutations '(1 2 3 4 5))

PERMUTATIONS

((1 2 3 4 5) (1 2 3 5 4) (1 2 4 3 5) (1 2 4 5 3) (1 2 5 3 4) (1 2 5 4 3)
 (1 3 2 4 5) (1 3 2 5 4) (1 3 4 2 5) (1 3 4 5 2) (1 3 5 2 4) (1 3 5 4 2)
 (1 4 2 3 5) (1 4 2 5 3) (1 4 3 2 5) (1 4 3 5 2) (1 4 5 2 3) (1 4 5 3 2)
 (1 5 2 3 4) (1 5 2 4 3) (1 5 3 2 4) (1 5 3 4 2) (1 5 4 2 3) (1 5 4 3 2)
 (2 1 3 4 5) (2 1 3 5 4) (2 1 4 3 5) (2 1 4 5 3) (2 1 5 3 4) (2 1 5 4 3)
 (2 3 1 4 5) (2 3 1 5 4) (2 3 4 1 5) (2 3 4 5 1) (2 3 5 1 4) (2 3 5 4 1)
 (2 4 1 3 5) (2 4 1 5 3) (2 4 3 1 5) (2 4 3 5 1) (2 4 5 1 3) (2 4 5 3 1)
 (2 5 1 3 4) (2 5 1 4 3) (2 5 3 1 4) (2 5 3 4 1) (2 5 4 1 3) (2 5 4 3 1)
 (3 1 2 4 5) (3 1 2 5 4) (3 1 4 2 5) (3 1 4 5 2) (3 1 5 2 4) (3 1 5 4 2)
 (3 2 1 4 5) (3 2 1 5 4) (3 2 4 1 5) (3 2 4 5 1) (3 2 5 1 4) (3 2 5 4 1)
 (3 4 1 2 5) (3 4 1 5 2) (3 4 2 1 5) (3 4 2 5 1) (3 4 5 1 2) (3 4 5 2 1)
 (3 5 1 2 4) (3 5 1 4 2) (3 5 2 1 4) (3 5 2 4 1) (3 5 4 1 2) (3 5 4 2 1)
 (4 1 2 3 5) (4 1 2 5 3) (4 1 3 2 5) (4 1 3 5 2) (4 1 5 2 3) (4 1 5 3 2)
 (4 2 1 3 5) (4 2 1 5 3) (4 2 3 1 5) (4 2 3 5 1) (4

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::PERMUTATIONS in DEFUN


In [114]:
;; exercise 4.3 
;; TODO:
(defparameter *dessert-ops*
  (list
    (op 
      'purchase-cake
      :preconds '(have-money)
      :add-list '(have-cake)
      :del-list '(have-money)
    )
    (op 
      'purchase-ice-cream
      :preconds '(have-money)
      :add-list '(have-ice-cream)
      :del-list '(have-money)
    )
    (op 
      'eat-cake
      :preconds '(have-cake)
      :add-list '(dessert-eatan have-ice-cream)  ;; gift from store
      :del-list '(have-cake)
    )
    (op 
      'eat-ice-cream
      :preconds '(have-ice-cream)
      :add-list '(dessert-eaten)
      :del-list '(have-ice-cream)
    )))

(use *dessert-ops*)
(debug1 :gps)
(gps '(have-money) '(dessert-eaten))

*DESSERT-OPS*

4

(:GPS)

((START) (EXECUTING PURCHASE-ICE-CREAM) (EXECUTING EAT-ICE-CREAM))


Goal:(DESSERT-EATEN)
Consider: (EAT-ICE-CREAM)
 Goal:(HAVE-ICE-CREAM)
 Consider: (PURCHASE-ICE-CREAM)
  Goal:(HAVE-MONEY)
 Action: (PURCHASE-ICE-CREAM)
Action: (EAT-ICE-CREAM)